In [1]:
import json
import requests 
from pprint import pprint
import time
import pandas as pd
from sqlalchemy import create_engine 
from config import api_key, username, password

# 1.0 Extract

## 1.1 Perform API call on RAWG database

In [2]:
# Specify the URL
url = "https://api.rawg.io/api/games?key=" + api_key + "&dates=2012-01-01,2016-12-31"

# Lists for 'released', 'rating', 'metacritic', 'esrb_rating.name'
names = []
ratings = []
released = []
metacritic =[]
esrb = []
game_id = []

while url != None:
    
    try:
        # Make request and store response
        response = requests.get(url).json()

        # Identify URL as the "next" url (i.e. the next page) in the response
        url = response['next']
    
        # Store attributes into lists
        for game in response['results']:
            names.append(game['name'])
            ratings.append(game['rating'])
            released.append(game['released'])
            metacritic.append(game['metacritic'])
            game_id.append(game['id'])      
        
            try:   
                esrb.append(game['esrb_rating']["name"])
   
            except: 
                esrb.append("None") 
            
        # If there the "next" page is not available, then break the loop        
        if url == 'null':
            break
    
    # Once 10000 rows are reached, the "next" page becomes invalid. Ignore keyerror.
    except KeyError:
        break 
    
    time.sleep(1)
    

## 1.2 Create dictionary from json results/lists

In [3]:
vg_dict = {'Game_ID':game_id, 'Name':names, 'Released':released,\
           'Ratings':ratings, 'Metacritic':metacritic, 'ESRB_rating':esrb}

## 1.3 Create dataframe from dictionary

In [4]:
new_vg_data_df = pd.DataFrame(vg_dict)
new_vg_data_df

,Game_ID,Name,Released,Ratings,Metacritic,ESRB_rating
0,3498,Grand Theft Auto V,2013-09-17,4.48,97.0,Mature
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.67,92.0,Mature
2,5286,Tomb Raider (2013),2013-03-05,4.07,86.0,Mature
3,802,Borderlands 2,2012-09-18,4.05,89.0,Mature
4,4291,Counter-Strike: Global Offensive,2012-08-21,3.59,81.0,Mature
...,...,...,...,...,...,...
9995,15015,Quell 4D,2016-10-14,0.00,NaN,None
9996,21527,Ski Park Tycoon,2015-02-03,0.00,NaN,None
9997,21434,Exodus,2014-07-11,0.00,NaN,None
9998,21057,Jane's Realty,2015-10-21,0.00,NaN,None


# 1.4 Import Video Games Sales CSV file

In [5]:
csv_file = "Resources/Video_Games_Sales_as_at_22_Dec_2016.csv"

In [6]:
videogame_sales_df = pd.read_csv(csv_file)
videogame_sales_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


# 2.0 Transform

## 2.1 Exploratory Data Analysis

## 2.1.1 Sort the RAWG Videogame and Videogame Sales dataframes to check the years and games included

In [7]:
videogame_sales_df.sort_values("Year_of_Release", ascending = True, inplace = True)
videogame_sales_df

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
262,Asteroids,2600,1980.0,Shooter,Atari,4.00,0.26,0.0,0.05,4.31,NaN,NaN,NaN,NaN,NaN,NaN
5360,Freeway,2600,1980.0,Action,Activision,0.32,0.02,0.0,0.00,0.34,NaN,NaN,NaN,NaN,NaN,NaN
546,Missile Command,2600,1980.0,Shooter,Atari,2.56,0.17,0.0,0.03,2.76,NaN,NaN,NaN,NaN,NaN,NaN
2650,Boxing,2600,1980.0,Fighting,Activision,0.72,0.04,0.0,0.01,0.77,NaN,NaN,NaN,NaN,NaN,NaN
4019,Ice Hockey,2600,1980.0,Sports,Activision,0.46,0.03,0.0,0.01,0.49,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16376,PDC World Championship Darts 2008,PSP,NaN,Sports,Oxygen Interactive,0.01,0.00,0.0,0.00,0.01,43.0,7.0,NaN,NaN,Oxygen Interactive,E10+
16409,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0.00,0.0,0.00,0.01,69.0,14.0,6.5,6.0,Midway,T
16452,Inversion,PC,NaN,Shooter,Namco Bandai Games,0.01,0.00,0.0,0.00,0.01,59.0,6.0,6.7,107.0,Saber Interactive,M
16462,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,Unknown,0.01,0.00,0.0,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
new_vg_data_df.sort_values("Released", ascending = True, inplace = True)
new_vg_data_df

,Game_ID,Name,Released,Ratings,Metacritic,ESRB_rating
4958,59026,The Light,2012-01-01,3.92,NaN,None
3817,41142,War of the Immortals,2012-01-01,0.00,NaN,None
3325,16757,1000 Amps,2012-01-01,3.67,NaN,None
4857,4503,NFL Blitz,2012-01-03,3.80,NaN,Everyone 10+
2839,28363,UFC Undisputed 3,2012-01-03,3.65,NaN,Teen
...,...,...,...,...,...,...
5769,15039,Mistwood Heroes,2016-12-30,0.00,NaN,None
2714,14942,Trick and Treat - Visual Novel,2016-12-30,2.88,NaN,None
7815,429,Divide,2016-12-31,0.00,NaN,Teen
6294,15160,Spooky Night,2016-12-31,0.00,NaN,None


## 2.2 Data Cleaning

## 2.2.1 Retain only columns needed from the Video Games Sales dataframe

In [9]:
videogame_sales_df = videogame_sales_df[['Name','Platform','Genre','Global_Sales']]
videogame_sales_df

,Name,Platform,Genre,Global_Sales
262,Asteroids,2600,Shooter,4.31
5360,Freeway,2600,Action,0.34
546,Missile Command,2600,Shooter,2.76
2650,Boxing,2600,Fighting,0.77
4019,Ice Hockey,2600,Sports,0.49
...,...,...,...,...
16376,PDC World Championship Darts 2008,PSP,Sports,0.01
16409,Freaky Flyers,GC,Racing,0.01
16452,Inversion,PC,Shooter,0.01
16462,Hakuouki: Shinsengumi Kitan,PS3,Adventure,0.01


## 2.2.2 Rename columns in the videogames sales dataframe to lower case

In [10]:
videogame_sales_df = videogame_sales_df.rename(columns = {'Global_Sales':'sales_units_in_millions',\
                                                          'Platform':'platform_id',\
                                                          'Name':'game_name', \
                                                          'Genre':'genre'})
videogame_sales_df

,game_name,platform_id,genre,sales_units_in_millions
262,Asteroids,2600,Shooter,4.31
5360,Freeway,2600,Action,0.34
546,Missile Command,2600,Shooter,2.76
2650,Boxing,2600,Fighting,0.77
4019,Ice Hockey,2600,Sports,0.49
...,...,...,...,...
16376,PDC World Championship Darts 2008,PSP,Sports,0.01
16409,Freaky Flyers,GC,Racing,0.01
16452,Inversion,PC,Shooter,0.01
16462,Hakuouki: Shinsengumi Kitan,PS3,Adventure,0.01


## 2.2.3 Rename columns in the metacritic dataframe to clearer column names

In [11]:
new_vg_data_df = new_vg_data_df.rename(columns = {'Game_ID':'game_id',\
                                        'Name':'game_name',\
                                        'Released':'release_date',\
                                        'Genre':'genre',\
                                        'Ratings':'rating',\
                                        'Metacritic':'metacritic',\
                                        'ESRB_rating':'esrb_rating'})
new_vg_data_df

,game_id,game_name,release_date,rating,metacritic,esrb_rating
4958,59026,The Light,2012-01-01,3.92,NaN,None
3817,41142,War of the Immortals,2012-01-01,0.00,NaN,None
3325,16757,1000 Amps,2012-01-01,3.67,NaN,None
4857,4503,NFL Blitz,2012-01-03,3.80,NaN,Everyone 10+
2839,28363,UFC Undisputed 3,2012-01-03,3.65,NaN,Teen
...,...,...,...,...,...,...
5769,15039,Mistwood Heroes,2016-12-30,0.00,NaN,None
2714,14942,Trick and Treat - Visual Novel,2016-12-30,2.88,NaN,None
7815,429,Divide,2016-12-31,0.00,NaN,Teen
6294,15160,Spooky Night,2016-12-31,0.00,NaN,None


## 2.2.4 Replace the NaN values in Metracritic to Not Rated

In [12]:
new_vg_data_df['metacritic'].fillna('Not Rated', inplace = True)
new_vg_data_df

,game_id,game_name,release_date,rating,metacritic,esrb_rating
4958,59026,The Light,2012-01-01,3.92,Not Rated,None
3817,41142,War of the Immortals,2012-01-01,0.00,Not Rated,None
3325,16757,1000 Amps,2012-01-01,3.67,Not Rated,None
4857,4503,NFL Blitz,2012-01-03,3.80,Not Rated,Everyone 10+
2839,28363,UFC Undisputed 3,2012-01-03,3.65,Not Rated,Teen
...,...,...,...,...,...,...
5769,15039,Mistwood Heroes,2016-12-30,0.00,Not Rated,None
2714,14942,Trick and Treat - Visual Novel,2016-12-30,2.88,Not Rated,None
7815,429,Divide,2016-12-31,0.00,Not Rated,Teen
6294,15160,Spooky Night,2016-12-31,0.00,Not Rated,None


## 2.2.5 Save RAWG API cleaned dataframes as csv/ json files

In [13]:
new_vg_data_df.to_json('Resources/rawg_vg_data.json')

In [14]:
new_vg_data_df.to_csv('Resources/rawg_vg_data.csv', index=False)

## 2.2.6 Merge the two dataframes

In [15]:
merged_df = new_vg_data_df.merge(videogame_sales_df, how='inner', on='game_name')

## 2.2.7 Locate the Game with the same Platform being shown twice

In [16]:
merged_df.loc[merged_df['game_id'] ==4273]

,game_id,game_name,release_date,rating,metacritic,esrb_rating,platform_id,genre,sales_units_in_millions
127,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,WiiU,Sports,0.23
128,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PSV,Sports,0.30
129,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PS3,Sports,2.56
130,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,X360,Sports,2.86
131,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,Wii,Sports,0.50
132,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PS3,Sports,0.01


## 2.2.8 Combine the sales unit for Game ID 4273 Platform PS3 by summing the two rows

In [17]:
merged_df.at[128, 'sales_units_in_millions'] = 2.57

In [18]:
merged_df.loc[merged_df['game_id'] ==4273]

,game_id,game_name,release_date,rating,metacritic,esrb_rating,platform_id,genre,sales_units_in_millions
127,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,WiiU,Sports,0.23
128,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PSV,Sports,2.57
129,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PS3,Sports,2.56
130,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,X360,Sports,2.86
131,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,Wii,Sports,0.50
132,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PS3,Sports,0.01


## 2.2.9 Drop the extra row

In [19]:
merged_df.drop(index=132, inplace = True)

In [20]:
merged_df.loc[merged_df['game_id'] ==4273]

,game_id,game_name,release_date,rating,metacritic,esrb_rating,platform_id,genre,sales_units_in_millions
127,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,WiiU,Sports,0.23
128,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PSV,Sports,2.57
129,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,PS3,Sports,2.56
130,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,X360,Sports,2.86
131,4273,Madden NFL 13,2012-08-28,2.68,Not Rated,Everyone 10+,Wii,Sports,0.50


## 2.2.10 Check each Game name has a Game ID

In [21]:
print(len(merged_df['game_id']))
print(len(merged_df['game_name']))

1232
1232


## 2.2.11 Save merged df to CSV

In [22]:
merged_df.to_csv('Resources/merged_games.csv', index=False)

## 2.3 Get data into separate dataframes to match tables

## 2.3.1 Create Game dataframe

In [23]:
game_df = merged_df[['game_id','game_name','release_date','genre']]
game_df = game_df.drop_duplicates()
game_df


,game_id,game_name,release_date,genre
0,4503,NFL Blitz,2012-01-03,Sports
2,28363,UFC Undisputed 3,2012-01-03,Action
4,12092,Age of Empires III: Complete Collection,2012-01-05,Strategy
5,27142,Rhythm Thief & the Emperor's Treasure,2012-01-19,Misc
6,18142,Oil Rush,2012-01-25,Strategy
...,...,...,...,...
1225,864,Dishonored 2,2016-11-10,Action
1228,9539,RollerCoaster Tycoon World,2016-11-16,Simulation
1229,572,EVE: Valkyrie,2016-11-17,Shooter
1230,10061,Watch Dogs 2,2016-11-28,Action


## 2.3.2 Create Platform dataframe

In [24]:
platform_df = merged_df[['platform_id']]
platform_df = platform_df.drop_duplicates()
platform_df

,platform_id
0,N64
1,PS
2,PS3
3,X360
4,PC
5,3DS
12,PSV
26,WiiU
33,PS2
36,DS


## 2.3.3 Create Rating dataframe

In [25]:
ratings_df = merged_df[['game_id','metacritic','rating']]
ratings_df = ratings_df.drop_duplicates()
ratings_df

,game_id,metacritic,rating
0,4503,Not Rated,3.80
2,28363,Not Rated,3.65
4,12092,81,4.13
5,27142,Not Rated,4.00
6,18142,67,2.38
...,...,...,...
1225,864,86,4.26
1228,9539,43,3.10
1229,572,69,0.00
1230,10061,79,3.86


## 2.3.4 Create ESRB Rating dataframe

In [26]:
esrb_df = merged_df[['game_id','esrb_rating']]
esrb_df = esrb_df.drop_duplicates()
esrb_df

,game_id,esrb_rating
0,4503,Everyone 10+
2,28363,Teen
4,12092,None
5,27142,None
6,18142,None
...,...,...
1225,864,Mature
1228,9539,None
1229,572,Teen
1230,10061,Mature


## 2.3.5 Create Sales dataframe

In [27]:
sales_df = merged_df[['game_id','platform_id','sales_units_in_millions']]
sales_df

,game_id,platform_id,sales_units_in_millions
0,4503,N64,1.06
1,4503,PS,1.50
2,28363,PS3,1.03
3,28363,X360,0.99
4,12092,PC,0.17
...,...,...,...
1228,9539,PC,0.02
1229,572,PS4,0.10
1230,10061,XOne,0.36
1231,10061,PS4,1.13


## 2.3.6 Create Game Platform dataframe

In [28]:
game_platform_df = merged_df[['platform_id','game_id']]
game_platform_df

,platform_id,game_id
0,N64,4503
1,PS,4503
2,PS3,28363
3,X360,28363
4,PC,12092
...,...,...
1228,PC,9539
1229,PS4,572
1230,XOne,10061
1231,PS4,10061


# 3.0 Load

## 3.1 Connect to local database


In [29]:
rds_connection_string = f"{username}:{password}@localhost:5432/game_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

## 3.2 Check for tables

In [30]:
engine.table_names()

[]

## 3.3 Export data from dataframes to tables

In [31]:
game_df.to_sql("game", con=engine, index=False, if_exists="append")

In [32]:
game_df.to_csv('Resources/games.csv', index=False)

In [33]:
platform_df.to_sql("platform", con=engine, index=False, if_exists="append")

In [34]:
platform_df.to_csv('Resources/platforms.csv', index=False)

In [35]:
esrb_df.to_sql("esrb_rating", con=engine, index=False, if_exists="append")

In [36]:
esrb_df.to_csv('Resources/esrb.csv', index=False)

In [37]:
sales_df.to_sql("sales_platform", con=engine, index=False, if_exists="append")

In [38]:
sales_df.to_csv('Resources/sales.csv', index=False)

In [39]:
ratings_df.to_sql("ratings", con=engine, index=False, if_exists="append")

In [40]:
ratings_df.to_csv('Resources/ratings.csv', index=False)

In [41]:
game_platform_df.to_sql("game_platform", con=engine, index=False, if_exists="append")

In [42]:
game_platform_df.to_csv('Resources/game_platforms.csv', index=False)

## 3.4 Confirm data has been added by querying tables
* NOTE: can also check using pgAdmin

In [43]:
pd.read_sql_query('select * from game', con=engine).head()

,game_id,game_name,release_date,genre
0,4503,NFL Blitz,2012-01-03,Sports
1,28363,UFC Undisputed 3,2012-01-03,Action
2,12092,Age of Empires III: Complete Collection,2012-01-05,Strategy
3,27142,Rhythm Thief & the Emperor's Treasure,2012-01-19,Misc
4,18142,Oil Rush,2012-01-25,Strategy
